# Predictive Analytics

## Task I

* build ML prototype that will predict if a question will be ansered in the next 2 hours
* model it as binary classification
* first prepare simple model with some basic features
* then try to improve it by adding some more features
* use random forest as a classifier
* for modelling consider only questions that have accepted answer
* if you run in local mode do not hyperparameter tuning since it may run to long

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, unix_timestamp, when, lit, length, array_sort, udf, desc
)

from pyspark.sql.types import (
    ArrayType, StructType, StructField, StringType, IntegerType
)

from pyspark.ml.tuning import CrossValidator, CrossValidatorModel, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.feature import VectorAssembler, Tokenizer, SQLTransformer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('Predictive Analytics I')
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

answers_input_path = os.path.join(project_path, 'data/answers')

questions_input_path = os.path.join(project_path, 'data/questions')

model_output_path = os.path.join(project_path, 'output/models/binary-classification')

<b>Load the data:</b>

In [ ]:
answersDF = (
    spark
    .read
    .option('path', answers_input_path)
    .load()
)

questionsDF = (
    spark
    .read
    .option('path', questions_input_path)
    .load()
)

<b>Add label to the dataset</b>

hint:
* join questions with answers
* compute response time using unix_timestamp
* use 'when' condition to compute the label

In [ ]:
# your code here

data_with_label = (
    questionsDF.alias('questions')
    .join(answersDF.alias('answers'), questionsDF['accepted_answer_id'] == answersDF['answer_id'])
    .select(
        col('questions.tags'),
        col('questions.creation_date').alias('question_time'),
        col('questions.title'),
        col('questions.body').alias('message'),
        col('answers.creation_date').alias('answer_time')
    )
    .withColumn('response_time', unix_timestamp('answer_time') - unix_timestamp('question_time'))
    .withColumn('label', when(col('response_time') <= 7200, lit(1)).otherwise(0))
).cache()

In [ ]:
data_with_label.count()

<b>Take a look at the distribution of classes</b>

In [ ]:
# your code here

(
    data_with_label
    .groupBy('label')
    .count()
).show()

<b>Add some basic features:</b>

hint:
* add feature 'title_complexity'
 * compute the length of the question title

In [ ]:
# your code here

data_with_basic_features = (
    data_with_label
    .withColumn('title_complexity', length('title'))
)

<b>Prepare data</b>

hint:
* split the data for training and testing using [randomSplit](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.randomSplit.html#pyspark.sql.DataFrame.randomSplit)

In [ ]:
# your code here

train_data, test_data = data_with_basic_features.randomSplit([0.7, 0.3], 24)

<b>Build the pipeline and train the model:</b>

hint:
* use: 
 * [VectorAssembler](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.VectorAssembler.html#pyspark.ml.feature.VectorAssembler)
 * [RandomForestClassifier](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html#pyspark.ml.classification.RandomForestClassifier)
 * [Pipeline](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.Pipeline.html#pyspark.ml.Pipeline)

In [ ]:
# your code here

features = ['title_complexity']

assembler = VectorAssembler(inputCols=(features), outputCol='features')

# Classifier:
rf = RandomForestClassifier(labelCol='label', featuresCol='features', seed=42)

pipeline = Pipeline(stages=[assembler, rf])

rf_model = pipeline.fit(train_data)

<b>Evaluate the model</b>

hint:
* use [BinaryClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html#pyspark.ml.evaluation.BinaryClassificationEvaluator) with areaUnderROC

In [ ]:
# your code here

evaluator = BinaryClassificationEvaluator(labelCol='label', metricName='areaUnderROC')

predictions = rf_model.transform(test_data)

evaluator.evaluate(predictions)

<b>Add more features</b>

hint:
* add features: 
    * 'question_size' number of words in the question body
     * use [Tokenizer](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Tokenizer.html#pyspark.ml.feature.Tokenizer) to split the text on words
     * use a [SQLTransformer](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.SQLTransformer.html#pyspark.ml.feature.SQLTransformer) to compute the size
    * you can try to create a feature using the tags

* train the model with this new pipeline
* evaluate the model
* see if the model improved

In [ ]:
sizeTrans = SQLTransformer(statement="SELECT *, size(words) AS message_size FROM __THIS__")

In [ ]:
features = ['title_complexity', 'message_size']

tokenizer = Tokenizer(inputCol='message', outputCol='words')

assembler = VectorAssembler(inputCols=(features), outputCol='features')

rf = RandomForestClassifier(labelCol='label', featuresCol='features', seed=42)

pipeline = Pipeline(stages=[tokenizer, sizeTrans, assembler, rf])

rf_model = pipeline.fit(train_data)

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol='label', metricName='areaUnderROC')

predictions = rf_model.transform(test_data)

evaluator.evaluate(predictions)

#### Note

* Similarly you could look for other features and try to improve the evaluation metric

<b>Hyperparameter tuning:</b>

hint:
* use [ParamGridBuilder](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html#pyspark.ml.tuning.ParamGridBuilder) to find optimal numTrees and optimal masDepth

Note:

If you run in local mode skip the hyperparameter tuning since it may run to long (more then hour).

In [ ]:
paramGrid = (
  ParamGridBuilder()
  .addGrid(rf.maxDepth, [3, 5, 8])
  .addGrid(rf.numTrees, [50, 100, 150])
  .build()
)

# cross_model = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid).fit(train_data)
# rf_model = cross_model.bestModel
# predictions = rf_model.transform(test_data)
# evaluator.evaluate(predictions)

### Save the model so you can use it later in some ml application

* use [write](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.PipelineModel.html#pyspark.ml.PipelineModel.write) on the [PipelineModel](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.PipelineModel.html#pyspark.ml.PipelineModel)

In [ ]:
# your code here

(
    rf_model
    .write()
    .overwrite()
    .save(model_output_path)
)